In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
import numpy as np
from scipy.spatial import distance

from tqdm.auto import tqdm
import pickle
import gc
import json
import h5py
import pandas as pd

from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

# == recnn ==
import sys
sys.path.append("../../")
import recnn


cuda = torch.device('cuda')
frame_size = 10
# https://drive.google.com/open?id=1xjVI4uVQGsQ7tjOJ3594ZXmAEC_6yX0e
meta = json.load(open('../../data/metadata.json'))
tqdm.pandas()

In [4]:
# https://drive.google.com/open?id=1kTyu05ZmtP2MA33J5hWdX8OyUYEDW4iI
movie_embeddings_key_dict = pickle.load(open('../../data/infos_pca128.pytorch', 'rb'))
movies_embeddings_tensor, \
key_to_id, id_to_key = recnn.data.make_items_tensor(movie_embeddings_key_dict)
# download ml20m dataset yourself
ratings = pd.read_csv('../../data/ml-20m/ratings.csv')
user_dict, users = recnn.data.prepare_dataset(ratings, key_to_id, frame_size)
del ratings
gc.collect()
clear_output(True)
clear_output(True)
print('Done!')

Done!


In [6]:
ddpg = recnn.models.Actor(1290, 128, 256).to(cuda)
td3 = recnn.models.Actor(1290, 128, 256).to(cuda)
ddpg.load_state_dict(torch.load('../../models/DDPG/ddpg_policy.pt'))
td3.load_state_dict(torch.load('../../models/TD3/td3_policy_1.pt'))

<All keys matched successfully>

In [7]:
test_users = users[-5000:]
batch_size = 25

def prepare_batch_wrapper(x):
    batch = recnn.data.prepare_batch_static_size(x, movies_embeddings_tensor, frame_size=frame_size)
    return batch

test_user_dataset = recnn.data.UserDataset(test_users, user_dict)
test_dataloader = DataLoader(test_user_dataset, batch_size=batch_size,
                        shuffle=True, num_workers=4,collate_fn=prepare_batch_wrapper)
test_batch = next(iter(test_dataloader))
state = test_batch[0].to(cuda)

In [9]:
genres_dict = {'Animation': 16,
 'Comedy': 35,
 'Family': 10751,
 'Adventure': 12,
 'Fantasy': 14,
 'Romance': 10749,
 'Drama': 18,
 'Action': 28,
 'Crime': 80,
 'Thriller': 53,
 'Horror': 27,
 'History': 36,
 'Science Fiction': 878,
 'Mystery': 9648,
 'War': 10752,
 'Music': 10402,
 'Documentary': 99,
 'Western': 37,
 'TV Movie': 10770,
 'Sport': 1,
 'Sci-Fi': 878,
 'Musical': 10402,
 'Short': 2,
 'News': 3,
 'Film-Noir': 4}
genres_dict = {v: k for k, v in genres_dict.items()}
del movie_embeddings_key_dict[0]

In [16]:
def rank(gen_action, metric):
    scores = []
    for i in movie_embeddings_key_dict.keys():
        scores.append([i, metric(movie_embeddings_key_dict[i], gen_action)])
    scores = list(sorted(scores, key = lambda x: x[1]))
    scores = scores[:10]
    ids = [i[0] for i in scores]
    for i in range(10):
        scores[i].extend([meta[str(scores[i][0])][key]  for key in ['title',
                                'genres', 'original_language', 'release_year', 'vote_average']])
        scores[i][3] = ' '.join([genres_dict[i] for i in scores[i][3]])

    indexes = ['id', 'score', 'title', 'genres', 'language', 'year', 'rating']
    table_dict = dict([(key,[i[idx] for i in scores]) for idx, key in enumerate(indexes)])
    table = pd.DataFrame(table_dict)
    return table

# DDPG

In [17]:
ddpg_action = ddpg(state)
# pick random action
ddpg_action = ddpg_action[np.random.randint(0, state.size(0), 1)[0]].detach().cpu().numpy()

In [18]:
from pandas.plotting import table
import subprocess
import matplotlib.pyplot as plt
%matplotlib inline

#from jupyterthemes import jtplot
#jtplot.style(theme='grade3')

rank(ddpg_action, distance.euclidean)

,id,score,title,genres,language,year,rating
0,89722,15.132885,A Horrible Way to Die,Horror Thriller Drama,en,2010,5.2
1,78329,15.220432,Reconstruction,Drama,ro,1970,6.3
2,101114,15.237413,The Net 2.0,Action Drama Thriller,en,2006,5.2
3,94310,15.355852,In The Hands Of The Gods,Documentary,en,2007,5.0
4,82834,15.384879,Elena and Her Men,Drama Comedy Romance,fr,1956,5.8
5,110555,15.430874,Love Meetings,Documentary,it,1965,8.4
6,112596,15.443581,Missing William,Drama,en,2014,3.5
7,92845,15.473968,Untamed Youth,Drama,en,1957,3.3
8,4543,15.478086,Shoot to Kill,Action Thriller,en,1988,6.9
9,71556,15.485146,How to Be a Serial Killer,Comedy Horror,en,2008,5.2


In [19]:
rank(ddpg_action, distance.cosine)

,id,score,title,genres,language,year,rating
0,116108,0.664427,The Secret Country: The First Australians Figh...,History Documentary,en,1986,0.0
1,55668,0.667947,Full of It,Comedy Family Drama,en,2007,5.3
2,118694,0.679762,In the Folds of the Flesh,Drama Horror Thriller,it,1970,5.9
3,75825,0.688651,The Card Player,Horror Mystery Thriller,it,2004,4.7
4,110555,0.690253,Love Meetings,Documentary,it,1965,8.4
5,82370,0.697973,The Master and Margaret,Drama Fantasy Romance,it,1972,5.6
6,109804,0.700611,God's Gun,Action Western,it,1975,4.4
7,91766,0.701198,Machine Gun McCain,Drama Action Crime,it,1969,6.9
8,107174,0.706098,La Rabbia,Documentary,it,1963,7.6
9,83186,0.707857,"Django Kill... If You Live, Shoot!",Action Western Horror,it,1967,5.6


In [20]:
rank(ddpg_action, distance.correlation) # looks similar to cosine

,id,score,title,genres,language,year,rating
0,116108,0.663871,The Secret Country: The First Australians Figh...,History Documentary,en,1986,0.0
1,55668,0.669312,Full of It,Comedy Family Drama,en,2007,5.3
2,118694,0.679486,In the Folds of the Flesh,Drama Horror Thriller,it,1970,5.9
3,110555,0.680988,Love Meetings,Documentary,it,1965,8.4
4,75825,0.687855,The Card Player,Horror Mystery Thriller,it,2004,4.7
5,82370,0.692053,The Master and Margaret,Drama Fantasy Romance,it,1972,5.6
6,107174,0.695473,La Rabbia,Documentary,it,1963,7.6
7,91766,0.699601,Machine Gun McCain,Drama Action Crime,it,1969,6.9
8,105252,0.700420,Conquest,Action Adventure Fantasy Horror,it,1983,5.6
9,109804,0.702303,God's Gun,Action Western,it,1975,4.4


In [21]:
rank(ddpg_action, distance.canberra)

,id,score,title,genres,language,year,rating
0,75825,78.142696,The Card Player,Horror Mystery Thriller,it,2004,4.7
1,6738,78.775182,Indiscretion of an American Wife,Drama Romance,it,1953,5.8
2,91167,79.445714,An Inconvenient Tax,Documentary,en,2011,7.0
3,116108,79.915607,The Secret Country: The First Australians Figh...,History Documentary,en,1986,0.0
4,68349,80.210120,The Life and Times of Allen Ginsberg,Documentary,en,2008,7.0
5,2687,80.322331,Tarzan,Adventure Animation Drama Family,en,1999,7.4
6,242,80.633498,Farinelli,History Drama,fr,1994,6.4
7,102477,80.710258,Tales of Vesperia: The First Strike,Action Adventure Animation Fantasy,ja,2009,6.1
8,96148,80.821004,The President's Man: A Line in the Sand,Thriller Drama Action,en,2002,4.5
9,80903,80.898816,One Million B.C.,Fantasy Horror,en,1940,6.1


In [22]:
rank(ddpg_action, distance.minkowski)

,id,score,title,genres,language,year,rating
0,89722,15.132885,A Horrible Way to Die,Horror Thriller Drama,en,2010,5.2
1,78329,15.220432,Reconstruction,Drama,ro,1970,6.3
2,101114,15.237413,The Net 2.0,Action Drama Thriller,en,2006,5.2
3,94310,15.355852,In The Hands Of The Gods,Documentary,en,2007,5.0
4,82834,15.384879,Elena and Her Men,Drama Comedy Romance,fr,1956,5.8
5,110555,15.430874,Love Meetings,Documentary,it,1965,8.4
6,112596,15.443581,Missing William,Drama,en,2014,3.5
7,92845,15.473968,Untamed Youth,Drama,en,1957,3.3
8,4543,15.478086,Shoot to Kill,Action Thriller,en,1988,6.9
9,71556,15.485146,How to Be a Serial Killer,Comedy Horror,en,2008,5.2


In [23]:
rank(ddpg_action, distance.chebyshev)

,id,score,title,genres,language,year,rating
0,5223,3.276079,Pauline & Paulette,Drama Comedy,nl,2001,6.8
1,100448,3.316839,Right America: Feeling Wronged,Documentary,en,2009,8.0
2,97163,3.321238,Shameless,Drama,no,2010,6.5
3,4227,3.330285,The Brothers,Comedy Drama Romance,en,2001,6.9
4,2066,3.344142,Out of the Past,Drama Mystery Thriller,en,1947,7.7
5,83006,3.357023,The Scientist,Drama Sci-Fi,en,2010,5.8
6,70366,3.357222,"Silent Night, Deadly Night Part 2",Comedy Horror,en,1987,4.4
7,8366,3.367303,Saved!,Comedy Drama,en,2004,6.4
8,86497,3.380644,15,Action Drama,ro,2005,7.0
9,82834,3.381478,Elena and Her Men,Drama Comedy Romance,fr,1956,5.8


In [24]:
rank(ddpg_action, distance.braycurtis)

,id,score,title,genres,language,year,rating
0,116108,0.698426,The Secret Country: The First Australians Figh...,History Documentary,en,1986,0.0
1,75825,0.701735,The Card Player,Horror Mystery Thriller,it,2004,4.7
2,55668,0.712931,Full of It,Comedy Family Drama,en,2007,5.3
3,124332,0.724734,The Snow Queen,Animation Fantasy Family,ru,2012,5.6
4,82991,0.726290,Stalin,Drama History War,en,1992,5.5
5,88356,0.727690,The Smurfs,Animation Family Adventure Comedy Fantasy,en,2011,5.6
6,109804,0.730035,God's Gun,Action Western,it,1975,4.4
7,96148,0.736963,The President's Man: A Line in the Sand,Thriller Drama Action,en,2002,4.5
8,87222,0.738577,Kung Fu Panda 2,Animation Family,en,2011,6.7
9,2687,0.741590,Tarzan,Adventure Animation Drama Family,en,1999,7.4


In [25]:
rank(ddpg_action, distance.cityblock)

,id,score,title,genres,language,year,rating
0,4543,130.148102,Shoot to Kill,Action Thriller,en,1988,6.9
1,81184,130.638275,A Short Film About John Bolton,Documentary,en,2003,5.5
2,97855,130.667328,Hitman's Run,Action,en,1999,5.8
3,97595,130.819168,Dive!,Documentary,en,2010,6.2
4,75825,131.329819,The Card Player,Horror Mystery Thriller,it,2004,4.7
5,89722,131.964432,A Horrible Way to Die,Horror Thriller Drama,en,2010,5.2
6,107117,133.557648,The Armstrong Lie,Documentary,en,2013,7.0
7,27713,133.602509,Bukowski - Born into This,Documentary,en,2003,7.7
8,72947,133.898605,Make the Yuletide Gay,Drama Comedy Romance,en,2009,6.1
9,78329,133.937180,Reconstruction,Drama,ro,1970,6.3


# TD3

In [26]:
td3_action = td3(state)
# pick random action
td3_action = td3_action[np.random.randint(0, 5000, 1)[0]].detach().cpu().numpy()

In [27]:
from pandas.plotting import table 
import matplotlib.pyplot as plt
%matplotlib inline

#from jupyterthemes import jtplot
#jtplot.style(theme='grade3')

rank(td3_action, distance.euclidean)

,id,score,title,genres,language,year,rating
0,4934,9.233231,The Exterminator,Action Thriller Crime Drama,en,1980,5.6
1,26402,9.594266,Harper Valley P.T.A.,Comedy,en,1978,7.0
2,5977,9.650244,Where’s Poppa?,Comedy,en,1970,6.8
3,33917,9.736309,The Member of the Wedding,Drama,en,1952,6.9
4,100003,9.744796,Up In Smoke,Comedy,en,1957,8.0
5,65738,9.818028,Revenge of the Nerds III: The Next Generation,Comedy,en,1992,4.1
6,110588,9.965463,The Iran Job,Documentary,en,2012,5.5
7,6320,10.022363,Scenes from a Mall,Comedy,en,1991,5.1
8,88823,10.024603,High Time,Comedy,en,1960,5.5
9,27850,10.063970,The Yes Men,Comedy Documentary,en,2003,7.2


In [28]:
rank(td3_action, distance.cosine)

,id,score,title,genres,language,year,rating
0,51111,0.644835,What Will You Do When You Catch Me?,Comedy,pl,1978,8.4
1,118274,0.655147,Legacy of Rage,Action Thriller,cn,1986,7.3
2,126032,0.656316,Betty Boop's Hallowe'en Party,Horror Animation Comedy,en,1933,6.3
3,100003,0.662870,Up In Smoke,Comedy,en,1957,8.0
4,127286,0.667584,Mentor,Drama,en,2006,10.0
5,3429,0.678949,Creature Comforts,Animation Comedy Family,en,1989,7.2
6,79872,0.688852,Chicago,Comedy Drama Crime,en,1927,5.0
7,48059,0.693100,A Boy Named Charlie Brown,Animation Comedy Family,en,1969,6.6
8,8827,0.695158,Bill Cosby: Himself,Comedy,en,1983,7.4
9,25752,0.695485,The Freshman,Comedy,en,1925,7.0


In [29]:
rank(td3_action, distance.correlation) # looks similar to cosine

,id,score,title,genres,language,year,rating
0,51111,0.646248,What Will You Do When You Catch Me?,Comedy,pl,1978,8.4
1,118274,0.655884,Legacy of Rage,Action Thriller,cn,1986,7.3
2,127286,0.658209,Mentor,Drama,en,2006,10.0
3,100003,0.659154,Up In Smoke,Comedy,en,1957,8.0
4,126032,0.661668,Betty Boop's Hallowe'en Party,Horror Animation Comedy,en,1933,6.3
5,3429,0.672376,Creature Comforts,Animation Comedy Family,en,1989,7.2
6,79872,0.690787,Chicago,Comedy Drama Crime,en,1927,5.0
7,48059,0.693267,A Boy Named Charlie Brown,Animation Comedy Family,en,1969,6.6
8,8827,0.697419,Bill Cosby: Himself,Comedy,en,1983,7.4
9,25752,0.699294,The Freshman,Comedy,en,1925,7.0


In [30]:
rank(td3_action, distance.canberra)

,id,score,title,genres,language,year,rating
0,65709,75.206775,John Waters: This Filthy World,Documentary Comedy History,en,2006,8.0
1,58895,76.333573,Black Peter,Drama Comedy,cs,1964,6.9
2,32834,76.771047,Macbeth,Drama,en,1948,7.1
3,84680,77.187206,My Foolish Heart,,en,1950,7.6
4,90903,77.378370,Mahjong,Comedy Drama,zh,1996,8.3
5,105595,78.680218,Onnenpeli,,en,1965,6.0
6,31549,78.756197,Fata Morgana,Documentary,de,1971,6.7
7,121677,78.769797,The Spiritual Boxer,Comedy Action,zh,1975,7.1
8,48059,78.860198,A Boy Named Charlie Brown,Animation Comedy Family,en,1969,6.6
9,3678,78.897593,The Man with the Golden Arm,Crime Drama Romance,en,1955,7.1


In [31]:
rank(td3_action, distance.minkowski)

,id,score,title,genres,language,year,rating
0,4934,9.233231,The Exterminator,Action Thriller Crime Drama,en,1980,5.6
1,26402,9.594266,Harper Valley P.T.A.,Comedy,en,1978,7.0
2,5977,9.650244,Where’s Poppa?,Comedy,en,1970,6.8
3,33917,9.736309,The Member of the Wedding,Drama,en,1952,6.9
4,100003,9.744796,Up In Smoke,Comedy,en,1957,8.0
5,65738,9.818028,Revenge of the Nerds III: The Next Generation,Comedy,en,1992,4.1
6,110588,9.965463,The Iran Job,Documentary,en,2012,5.5
7,6320,10.022363,Scenes from a Mall,Comedy,en,1991,5.1
8,88823,10.024603,High Time,Comedy,en,1960,5.5
9,27850,10.063970,The Yes Men,Comedy Documentary,en,2003,7.2


In [32]:
rank(td3_action, distance.chebyshev)

,id,score,title,genres,language,year,rating
0,108134,2.043516,The Lizard,Action Comedy,fa,2004,6.0
1,97740,2.105000,"3, 2, 1... Frankie Go Boom",Comedy,en,2012,5.6
2,48264,2.140455,Vixen!,Drama,en,1968,5.6
3,97115,2.142834,El tren de la memoria,,en,2005,6.0
4,70826,2.161969,Summer School,Horror,en,2006,5.0
5,128968,2.164808,Stitch! The Movie,Comedy Adventure Family Animation,en,2003,6.4
6,119685,2.174955,Alyce Kills,Horror Thriller,en,2011,5.6
7,122104,2.175419,Kevin Hart: Seriously Funny,Comedy,en,2010,7.3
8,53187,2.190536,Beauty in Trouble,Comedy Drama,cs,2006,6.9
9,1324,2.193098,Amityville: Dollhouse,Horror,en,1996,3.9


In [33]:
rank(td3_action, distance.braycurtis)

,id,score,title,genres,language,year,rating
0,51111,0.674632,What Will You Do When You Catch Me?,Comedy,pl,1978,8.4
1,100003,0.690908,Up In Smoke,Comedy,en,1957,8.0
2,90903,0.692813,Mahjong,Comedy Drama,zh,1996,8.3
3,48059,0.706261,A Boy Named Charlie Brown,Animation Comedy Family,en,1969,6.6
4,79872,0.709632,Chicago,Comedy Drama Crime,en,1927,5.0
5,5142,0.713405,The Firemen's Ball,Comedy Drama,cs,1967,7.2
6,118274,0.715247,Legacy of Rage,Action Thriller,cn,1986,7.3
7,25752,0.718239,The Freshman,Comedy,en,1925,7.0
8,110415,0.729746,Breaking the Silence: Truth and Lies in the Wa...,Documentary,en,2003,7.8
9,121677,0.730210,The Spiritual Boxer,Comedy Action,zh,1975,7.1


In [34]:
rank(td3_action, distance.cityblock)

,id,score,title,genres,language,year,rating
0,26402,81.112930,Harper Valley P.T.A.,Comedy,en,1978,7.0
1,4934,85.056664,The Exterminator,Action Thriller Crime Drama,en,1980,5.6
2,33917,86.246506,The Member of the Wedding,Drama,en,1952,6.9
3,100559,86.785416,The Match King,Drama,en,1932,7.0
4,82848,86.861526,One Week,Comedy,en,1920,7.9
5,27850,86.983826,The Yes Men,Comedy Documentary,en,2003,7.2
6,88823,87.571602,High Time,Comedy,en,1960,5.5
7,65738,87.697380,Revenge of the Nerds III: The Next Generation,Comedy,en,1992,4.1
8,100003,87.842415,Up In Smoke,Comedy,en,1957,8.0
9,110588,88.119072,The Iran Job,Documentary,en,2012,5.5


# BCQ

In [36]:
perturbator = recnn.models.bcqPerturbator(1290, 128, 256).to(cuda)
generator = recnn.models.bcqGenerator(1290, 128, 512).to(cuda)

perturbator.load_state_dict(torch.load('../../models/BCQ/bcq_policy.pt'))
generator.load_state_dict(torch.load('../../models/BCQ/bcq_vae.pt'))

<All keys matched successfully>

In [37]:
sampled_actions = generator.decode(state)
perturbed_actions= perturbator(state, sampled_actions)

In [38]:
bcq_action = perturbed_actions[np.random.randint(0,
                               sampled_actions.size(0), 1)[0]].detach().cpu().numpy()

In [39]:
rank(bcq_action, distance.euclidean)

,id,score,title,genres,language,year,rating
0,6745,9.515295,Shrimp on the Barbie,Comedy Romance,en,1990,5.9
1,1629,9.713503,The Matchmaker,Comedy Romance,en,1997,6.3
2,31047,9.857168,Jinxed!,Comedy,en,1982,4.3
3,5513,9.858193,Martin Lawrence Live: Runteldat,Comedy,en,2002,5.0
4,87683,9.869917,An Unexpected Love,Drama TV Movie,en,2003,5.9
5,47423,9.888137,Half Nelson,Drama,en,2006,6.7
6,78574,9.922365,Winter's Bone,Drama,en,2010,6.9
7,87270,9.933637,The Other Woman,Drama,en,2010,6.2
8,106897,9.955842,Stuck Between Stations,Drama Romance,en,2011,6.2
9,88823,9.963136,High Time,Comedy,en,1960,5.5


In [40]:
rank(bcq_action, distance.cosine)

,id,score,title,genres,language,year,rating
0,106696,0.673634,Frozen,Animation Adventure Family,en,2013,7.3
1,42794,0.691185,Buried Alive,TV Movie Horror Thriller,en,1990,6.4
2,6615,0.691691,Freddy vs. Jason,Horror,en,2003,5.8
3,6745,0.693560,Shrimp on the Barbie,Comedy Romance,en,1990,5.9
4,87222,0.697007,Kung Fu Panda 2,Animation Family,en,2011,6.7
5,112776,0.698551,Bloodline,Horror,it,2011,1.0
6,78062,0.705234,Puppet Master vs Demonic Toys,Horror,en,2004,3.1
7,113674,0.709200,Proxy,Drama Horror Thriller,en,2013,5.9
8,40339,0.710282,Chicken Little,Animation Family Comedy,en,2005,5.8
9,95105,0.711013,Madagascar 3: Europe's Most Wanted,Animation Family Comedy,en,2012,6.5


In [41]:
rank(bcq_action, distance.correlation)

,id,score,title,genres,language,year,rating
0,106696,0.673739,Frozen,Animation Adventure Family,en,2013,7.3
1,42794,0.690291,Buried Alive,TV Movie Horror Thriller,en,1990,6.4
2,6615,0.691004,Freddy vs. Jason,Horror,en,2003,5.8
3,6745,0.693545,Shrimp on the Barbie,Comedy Romance,en,1990,5.9
4,87222,0.696411,Kung Fu Panda 2,Animation Family,en,2011,6.7
5,112776,0.698540,Bloodline,Horror,it,2011,1.0
6,78062,0.705381,Puppet Master vs Demonic Toys,Horror,en,2004,3.1
7,40339,0.707243,Chicken Little,Animation Family Comedy,en,2005,5.8
8,113674,0.708868,Proxy,Drama Horror Thriller,en,2013,5.9
9,95105,0.709236,Madagascar 3: Europe's Most Wanted,Animation Family Comedy,en,2012,6.5


In [42]:
rank(bcq_action, distance.canberra)

,id,score,title,genres,language,year,rating
0,129456,78.408064,The Legend of Bloody Jack,Horror,en,2007,4.4
1,833,78.562517,High School High,Comedy,en,1996,5.4
2,101662,78.929579,Three Way,Crime Drama Thriller,en,2004,4.3
3,97462,79.319606,Hoodwinked Too! Hood VS. Evil,Comedy Animation Family,en,2011,5.0
4,85790,79.422699,Meek's Cutoff,Drama Western,en,2010,6.5
5,45576,79.646512,Sinbad of the Seven Seas,Adventure Family,en,1989,3.7
6,73488,79.841134,Blood: The Last Vampire,Action Adventure Horror Sci-Fi Thriller,en,2009,5.1
7,7051,79.941488,What's New Pussycat?,Comedy Romance,en,1965,6.3
8,5621,79.946517,The Tuxedo,Thriller Action Comedy Sci-Fi,en,2002,5.5
9,6178,80.201325,A Patch of Blue,Drama Romance,en,1965,8.0


In [43]:
rank(bcq_action, distance.minkowski)

,id,score,title,genres,language,year,rating
0,6745,9.515295,Shrimp on the Barbie,Comedy Romance,en,1990,5.9
1,1629,9.713503,The Matchmaker,Comedy Romance,en,1997,6.3
2,31047,9.857168,Jinxed!,Comedy,en,1982,4.3
3,5513,9.858193,Martin Lawrence Live: Runteldat,Comedy,en,2002,5.0
4,87683,9.869917,An Unexpected Love,Drama TV Movie,en,2003,5.9
5,47423,9.888137,Half Nelson,Drama,en,2006,6.7
6,78574,9.922365,Winter's Bone,Drama,en,2010,6.9
7,87270,9.933637,The Other Woman,Drama,en,2010,6.2
8,106897,9.955842,Stuck Between Stations,Drama Romance,en,2011,6.2
9,88823,9.963136,High Time,Comedy,en,1960,5.5


In [44]:
rank(bcq_action, distance.chebyshev)

,id,score,title,genres,language,year,rating
0,78039,1.942187,Blue Valentine,Drama Romance,en,2010,6.9
1,106897,2.047636,Stuck Between Stations,Drama Romance,en,2011,6.2
2,6295,2.053344,Chasing Papi,Comedy Romance,en,2003,6.0
3,94192,2.059334,Devil's Diary,Drama Horror,en,2007,4.5
4,129011,2.066347,Eddie Izzard: Glorious,Comedy,en,1997,6.8
5,6788,2.088190,Angie,Comedy Drama Romance,en,1994,5.4
6,79226,2.093588,The Good Guy,Comedy Romance,en,2009,5.6
7,3916,2.101965,Remember the Titans,Drama,en,2000,7.6
8,2069,2.109872,The Trip to Bountiful,Drama,en,1985,7.4
9,50594,2.113705,Daisies,Comedy,cs,1966,7.7


In [45]:
rank(bcq_action, distance.braycurtis)

,id,score,title,genres,language,year,rating
0,6745,0.712504,Shrimp on the Barbie,Comedy Romance,en,1990,5.9
1,97462,0.721611,Hoodwinked Too! Hood VS. Evil,Comedy Animation Family,en,2011,5.0
2,101662,0.727641,Three Way,Crime Drama Thriller,en,2004,4.3
3,40339,0.730188,Chicken Little,Animation Family Comedy,en,2005,5.8
4,63436,0.730517,Saw V,Horror Thriller Crime,en,2008,6.1
5,100577,0.733169,The Angel Levine,Drama,en,1970,0.0
6,90576,0.733863,What's Your Number?,Comedy Romance,en,2011,6.4
7,54144,0.735894,The Last Time,Drama Romance,en,2006,5.5
8,87270,0.739809,The Other Woman,Drama,en,2010,6.2
9,3640,0.741412,A King in New York,Comedy Drama,en,1957,7.0


In [46]:
rank(bcq_action, distance.cityblock)

,id,score,title,genres,language,year,rating
0,6745,83.819412,Shrimp on the Barbie,Comedy Romance,en,1990,5.9
1,833,86.041580,High School High,Comedy,en,1996,5.4
2,1629,86.602219,The Matchmaker,Comedy Romance,en,1997,6.3
3,87270,88.043610,The Other Woman,Drama,en,2010,6.2
4,47423,88.132385,Half Nelson,Drama,en,2006,6.7
5,975,88.555115,Something to Sing About,Comedy,en,1937,5.4
6,84557,88.556129,From Prada to Nada,Comedy Drama Romance,en,2011,5.1
7,87683,88.685097,An Unexpected Love,Drama TV Movie,en,2003,5.9
8,33917,88.829498,The Member of the Wedding,Drama,en,1952,6.9
9,88823,88.973022,High Time,Comedy,en,1960,5.5
